# EXTRACT 
## Fetching Data

In [1]:
""" Load in modules we will need """
import os
import gc
import numpy as np 
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime as dt
load_dotenv()
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#### "Reading" from Bronze Layer 

In [ ]:
"""
Retrieve all the csv file names from a given path

Parameters: 
path (string)- Path we are looking through

Returns: a list of .csv files
"""
def grab_files(path: str) -> list:
    return [file for file in os.listdir(path) if file.endswith(".csv")]

"""
Creates DataFrames for all data of a specific entity in "Bronze layer"

Parameters: 
path (string) - The "container" for the data 

Returns: a list of .csv files
"""
def gen_dfs(ospath = 'BRONZE_LAYER', path:str =''):    
    dfs = []
    try:
        data_dir = f'{os.environ[ospath]}{path + "/" if path != "" else path}'
        files = grab_files(data_dir)
        for f in files:
            df = pd.read_csv(data_dir+f)
            df.name = f.split('__')[1]
            dfs.append(df)
    except FileNotFoundError:
        print("Error with pathing")
    except:
        print('Error Occurred while extracting data from the file')
    return dfs        #create the dataframe


# TRANSFORM
## Cleaning Functions

In [ ]:
"""
Will Attempt to Drop all declared columns from a DataFrame

Parameters: 
df - a singular Data Frame
cols - a list of strings containing column attributes to be dropped

Returns: Nothing
"""
def drop_dfcols(df, cols:list):
    try:
        df.drop(columns = cols, inplace = True)
    except KeyError:
        print(f'fail to remove{cols}')

"""
Will Attempt to Drop all declared columns from a List of DataFrame

Parameters: 
df - a List of Data Frame
cols - a list of strings containing column attributes to be dropped 

Returns: Nothing
"""
def drop_dfs_col(dfs, cols: list):
    try:
        for df in dfs:
            drop_dfcols(df,cols) 
    except KeyError:
        print('no columns left')
    


In [ ]:
"""
Remove duplicate records from the DataFrame
"""
def rm_dupe(df):
    df.drop_duplicates(inplace= True)

"""
Drops the columns that have null vals
or we fill them will default values if we want
"""
def rm_nulls(df, col, default_val = None):
    if default_val:
        df[col].fillna(default_val, inplace = True)
    else:
        df.dropna(subset =[col], inplace= True) 

"""
Checks to see if any column of a DataFrame contains Nulls
-best to do this after dropping useless columns [faster]
"""
def cols_with_null(df):
    has_null = []
    for col in df.columns:
        print(f'{col} : {df[col].isnull().values.any()}')
        if df[col].isnull().values.any():
            has_null.append(col)
    return has_null

"""
Parameters:
dfs: List of DataFrames 
Return: Merged DataFrames 
"""
def merge_dfs(dfs: list):
    name_attr = dfs[0].name if hasattr(dfs[0], "name") else None
    dfs = pd.concat(dfs, ignore_index=True)        #put all the data into one table
    if name_attr:
        dfs.name = name_attr
    return dfs

In [ ]:
"""
Checks to see if a val is of type : etype

Parameters:
etype: Type 

Returns:
Boolean if it is that type
"""
def is_type(val,  etype):
    try:
        etype(val)
        return True
    except (ValueError, TypeError):
        return False
    
"""
Removes all the records that have an invalid type

Parameters:
header_df : DataFrame
col : column name to check
etype: Type 

Returns:
DataFrame with records that have valid type
"""
def rm_inval_type(df, col, etype):
    name_attr = df.name if hasattr(df, "name") else None
    df = df[df[col].apply(lambda val: is_type(val,etype))]
    if name_attr:
        df.name = name_attr
    return df

In [ ]:
"""
Removes outliers (using Interquartile range) from a given column
Parameters:
df: DataFrame
col: column to check for outliers
Returns: DataFrame with outliers removed
"""
def IQR_outlier(df, col: str):
    name_attr = df.name if hasattr(df, "name") else None
    Q1 = df[col].quantile(0.25)   
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    # non_outliers = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    df_rm_outliers = df[~((df[col] < lower_bound) | (df[col] > upper_bound))]
    if name_attr:
        df_rm_outliers.name = name_attr
    return df_rm_outliers


"""
Removes outliers (using Z-score) from a given column
Parameters:
df: DataFrame
col: column to check for outliers
deviance: outlier bounds
Returns: DataFrame with outliers removed
"""
def zscore(df, col:str, deviance:int):
    col_mean = df[col].mean()
    col_std = df[col].std()
    return df[abs((df[col] - col_mean)/ col_std) <= deviance]

# LOAD
### Save into Currated layer [Silver]

In [ ]:
"""
Will file "modded_{filename}.csv"
in curated-layer directory.
If path does not exist, will create it for you.

Parameters: 
df - a Data Frame

Returns: Nothing
"""
def df2csv(df):
    year = df.name[-4:]
    type = df.name[:-5]
    path = f'../data_set/CBP_AMS/curated-layer/{year}/{type}/'          
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_csv(path + f'modded_{df.name}.csv', sep='|' , index=False) 

    """
Will file "modded_{filename}.parquet"
in curated-layer directory.
If path does not exist, will create it for you.

Parameters: 
df - a Data Frame

Returns: Nothing
"""
def df2parquet(df):
    year = df.name[-4:]
    type = df.name[:-5]
    path = f'../data_set/CBP_AMS/curated-layer/{year}/{type}/'          
    if not os.path.exists(path):
        os.makedirs(path)
    df.to_parquet(path + f'modded_{df.name}.parquet', engine='pyarrow',index=False) 

##### For Current Use Case : We can reduce data overhead

In [ ]:
rm_billgen = ['house_bol_number','sub_house_bol_number', 'bill_type_code',
              'manifest_number', 'trade_update_date', 'run_date']

rm_container = ['seal_number_1', 'seal_number_2',
       'equipment_description_code','container_type', 'load_status']

rm_header  = ['carrier_code', 'vessel_country_code', 'vessel_name',
       'foreign_port_of_lading_qualifier', 'manifest_quantity', 'manifest_unit', 
       'measurement', 'measurement_unit', 'record_status_indicator',
       'place_of_receipt', 'port_of_destination', 'foreign_port_of_destination_qualifier', 
       'foreign_port_of_destination', 'conveyance_id_qualifier', 'conveyance_id', 
       'in_bond_entry_type', 'mode_of_transportation', 'secondary_notify_party_1',
       'secondary_notify_party_2', 'secondary_notify_party_3', 'secondary_notify_party_4', 
       'secondary_notify_party_5', 'secondary_notify_party_6', 'secondary_notify_party_7',
       'secondary_notify_party_8', 'secondary_notify_party_9', 'secondary_notify_party_10']

rm_tariff = ['description_sequence_number','harmonized_number', 'harmonized_weight_unit','harmonized_weight']

rm_cargodesc = [ 'description_sequence_number','description_text']


# ----------HEADER--------------

#### Cleaning Header Data

In [ ]:
"""
Convert the weight column to a universal weight unit measurement

Parameters:
header_df: DataFrame

Returns:
None
"""
def universal_unit(header_df):
    unit_weight = {
        'Kilograms': 1.0,
        'Pounds': 0.453592, 
        'Metric Ton': 1000.0, 
        'Long Ton': 1016.04691, 
        'Measurement Ton': 1.01604691
    }
    # Convert weight to kg based on weight_unit
    for unit, scale in unit_weight.items():
        header_df.loc[header_df['weight_unit'] == unit, 'weight'] *= scale
        header_df['weight_unit'] = 'Kilograms'


def is_date(val):
    try:
        dt.strptime(val, '%Y-%m-%d')
        return True
    except (ValueError, TypeError):
        return False

def rm_inval_date(df,col):
    name_attr = df.name if hasattr(df, "name") else None
    df = df[df[col].apply(is_date)]
    if name_attr:
        df.name = name_attr
    return df

"""
removes the records that have dates that are not in the scope of this year
"""
def date_outlier(df):
    name_attr = df.name if hasattr(df, "name") else None
    year = int(df.name[-4:])
    df['estimated_arrival_date'] = pd.to_datetime(df['estimated_arrival_date'], format='%Y-%m-%d')
    df['actual_arrival_date'] = pd.to_datetime(df['actual_arrival_date'], format='%Y-%m-%d')
    df = df[df['actual_arrival_date'].dt.year <= year]   # we are not in the future!!!
    df = df[(abs(df['estimated_arrival_date'].dt.year - year) <= 5)]
    df = df[(abs(df['actual_arrival_date'].dt.year - year) <= 5)]
    if name_attr:
        df.name = name_attr
    return df
    


#### Load in HEADER Data

In [ ]:
# Currated layer only requires : container |  header  |  tariff   |   billgen   
dfs = gen_dfs(path='header')
dfs[0]

In [ ]:
# Drop columns we will not be using
drop_dfs_col(dfs, rm_header)

In [ ]:
# Merge the dataframes into one 
dfs = merge_dfs(dfs)

In [ ]:
# Drop the duplicate records
rm_dupe(dfs)



#### Removal of any irregular data in the record

In [ ]:
# Check to make sure there are no null values 😀
for col in cols_with_null(dfs):
    rm_nulls(dfs, col)
# Header does not need to deal with any Nulls 🎉

In [ ]:
# check the type of weight columns 
dfs = rm_inval_type(dfs, 'weight', int)

In [ ]:
# check the type of port
dfs = rm_inval_type(dfs, 'port_of_unlading', str)

In [ ]:
# check the type of port
dfs = rm_inval_type(dfs, 'foreign_port_of_lading', str)

In [ ]:
# check the type of unit
dfs = rm_inval_type(dfs, 'weight_unit', str)

In [ ]:
# Removes any invalid date in this col
dfs = rm_inval_date(dfs, 'estimated_arrival_date')

In [ ]:
# Removes any invalid date in this col
dfs = rm_inval_date(dfs, 'actual_arrival_date')

In [ ]:
# Remove out the Outliers
dfs = IQR_outlier(dfs, 'weight')

In [ ]:
# remove any date outliers 
dfs = date_outlier(dfs)

In [ ]:
# Clean the weight unit to a singular unit ( Kilogram )
universal_unit(dfs)

In [ ]:
# Cleaned up Header Data -> Store into "Silver Layer" 
df2parquet(dfs)

# ----------BILL GEN--------------

In [ ]:
# Currated layer only requires : container |  header  |  tariff   |   billgen   
dfs = gen_dfs(path='billgen')
dfs[0]

In [ ]:
# Drop columns we will not be using
drop_dfs_col(dfs, rm_billgen)

In [ ]:
# Merge the dataframes into one  
dfs = merge_dfs(dfs)

In [ ]:
# Drop the duplicate records  
rm_dupe(dfs)

In [ ]:
# remove any null values
for col in cols_with_null(dfs):
    rm_nulls(dfs, col)

In [ ]:
# Cleaned up Bill Gen Data -> Store into "Silver Layer" 
df2parquet(dfs)

# ----------CONTAINER--------------

In [ ]:
def rm_box_outlier(df):
    name_attr = df.name if hasattr(df, "name") else None
    df = df[(df['container_length'] != 0) & (df['container_width'] != 0) & (df['container_height'] != 0)]
    if name_attr:
        df.name = name_attr
    return df

In [ ]:
# Currated layer only requires : container |  header  |  tariff   |   billgen   
dfs = gen_dfs(path='container')
dfs[0]

In [ ]:
# Drop columns we will not be using
drop_dfs_col(dfs, rm_container)

In [ ]:
# Merge the dataframes into one    30635677 rows × 6 columns   27234627 rows × 6 columns
dfs = merge_dfs(dfs)

In [ ]:
# Drop the duplicate records  
rm_dupe(dfs)

In [ ]:
# remove any null values
for col in cols_with_null(dfs):
    rm_nulls(dfs, col)

In [ ]:
# Removes any invalid length in this col
dfs = rm_inval_type(dfs, 'container_length', int)

In [ ]:
# Removes any invalid height in this col
dfs = rm_inval_type(dfs, 'container_height', int)

In [ ]:
# Removes any invalid width in this col
dfs = rm_inval_type(dfs, 'container_width', int)

In [ ]:
dfs = rm_box_outlier(dfs)

In [ ]:
# grouped_counts = dfs.groupby(['container_length', 'container_height', 'container_width']).size().reset_index(name='counts')


In [ ]:
# grouped_counts.sort_values('counts')

In [ ]:
df2parquet(dfs)

# ------Tariff + Cargo Desc------

#### Load in Tariff

In [ ]:
def rm_cgd_outliers(df):
    name_attr = df.name if hasattr(df, "name") else None
    df = df[(df['piece_count'] != 0)]
    if name_attr:
        df.name = name_attr
    return df

def rm_tariff_outliers(df):
    name_attr = df.name if hasattr(df, "name") else None
    df = df[(df['harmonized_value'] > 0)]
    if name_attr:
        df.name = name_attr
    return df

In [ ]:
# Currated layer only requires : container |  header  |  tariff   |   billgen   |   cargodesc
tdfs = gen_dfs(path='tariff')
tdfs[0]

In [ ]:
# Drop columns we will not be using
drop_dfs_col(tdfs, rm_tariff)

In [ ]:
# Merge the dataframes into one   
tdfs = merge_dfs(tdfs)

In [ ]:
# Drop the duplicate records  
rm_dupe(tdfs)

In [ ]:
# remove any null values
for col in cols_with_null(tdfs):
    rm_nulls(tdfs, col)

In [ ]:
# check the type of piece
tdfs = rm_inval_type(tdfs,'harmonized_value', int)

In [ ]:
# remove value outliers
tdfs = rm_tariff_outliers(tdfs)

In [ ]:
tdfs = IQR_outlier(tdfs, 'harmonized_value')

#### Load in Cargo Desc

In [ ]:
# Currated layer only requires : container |  header  |  tariff   |   billgen   |   cargodesc
dfs = gen_dfs(path='cargodesc')
dfs[0]

In [ ]:
# Drop columns we will not be using
drop_dfs_col(dfs, rm_cargodesc)

In [ ]:
# Merge the dataframes into one    30635677 rows × 6 columns   27234627 rows × 6 columns
dfs = merge_dfs(dfs)

In [ ]:
# Drop the duplicate records  
rm_dupe(dfs)

In [ ]:
# remove any null values
for col in cols_with_null(dfs):
    rm_nulls(dfs, col)

In [ ]:
# check the type of piece
dfs = rm_inval_type(dfs,'piece_count', int)

In [ ]:
dfs = rm_cgd_outliers(dfs)
# done cleaning cargo desc

In [ ]:
dfs

In [ ]:
tdfs

In [ ]:
mdfs = pd.merge(dfs,tdfs, on=['identifier', 'container_number'])
mdfs.name = tdfs.name[:-5] + '_' + dfs.name[:-5] + '_' + tdfs.name[-4:]

In [ ]:
mdfs['tot_cost'] = mdfs['piece_count'] * mdfs['harmonized_value']

In [ ]:
mdfs.sort_values('container_number')

In [ ]:
df2parquet(mdfs)

# Store Data Into DB

In [2]:
import pyodbc
from sqlalchemy import create_engine
import urllib

def grab_files(path: str) -> list:
    file=[file for file in os.listdir(path) if file.endswith(".parquet")]
    return file[0]

def load_pq_df(model):
    path = f'{os.environ['SILVER_LAYER']}{model}/'
    file = grab_files(path)
    df = pd.read_parquet(path + file)
    return df

In [3]:
params = urllib.parse.quote_plus(r'DRIVER={ODBC Driver 17 for SQL Server};'
                                 r'SERVER=DESKTOP-N9SA336;'
                                 r'DATABASE=BastionSLA;'
                                 r'Trusted_Connection=yes;')
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

In [5]:
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};Server={DESKTOP-N9SA336};Database=;Trusted_Connection=yes;')
cursor = conn.cursor() # Cursors are database level objects that let your query a database multiple times - Think of it as a pointer to a row


In [6]:
# Create a DB as Silver Layer Storage
db_name = "BastionSLA"
cursor.execute(f"""
        IF NOT EXISTS (SELECT * FROM sys.databases WHERE name = '{db_name}')
        BEGIN
            CREATE DATABASE {db_name}
        END
        """)

In [7]:
cursor.execute('use BastionSLA')

In [63]:
df = load_pq_df('billgen')

In [68]:
table_name = 'billgen'
try:
    create_table_query = f"""
                    IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
                    BEGIN
                        CREATE TABLE {table_name} (
                            identifier BIGINT NOT NULL,
                            master_bol_number VARCHAR(255),
                            voyage_number VARCHAR(50)
                        )
                    END
                    """
    cursor.execute(create_table_query)
    conn.commit()
    print('Table has been created')
except:
    print('Alrdy exist or something went wrong')

Table has been created


In [69]:
# WRITE BILLGEN to DATABASE
df.to_sql(table_name, engine, if_exists='append', index=False, schema='dbo')

412

In [65]:
df

,identifier,master_bol_number,voyage_number
0,201901010,OOLU2593478240,091E
1,201901011,MOLU13903770302,033E
2,201901012,APLU711051802,00025
3,201901013,APLU711051813,00025
4,201901014,APLU711051920,00025
...,...,...,...
19336844,2019123164866,APLUSH1S01538424,0DB4P
19336845,2019123164867,CMDUMXO0470425,0VB4O
19336846,2019123164868,HLCUEUR1912AXQH2,044E
19336847,2019123164869,OOLU2110459500,041A


# USE CASE


In [ ]:
#Q1  Frequency for each type of service
